# FairVIC Supplementary Code
In this notebook you can find all of the supplementary code behind our method, FairVIC.

## Misc (Imports, Misc. functions, preprocessing)
This section is full of various imports, generic functions, or preprocessing steps for the data that is used. It is unimportant but skim through if you would like.

### Imports

In [ ]:
!pip install ucimlrepo
!pip install aif360

In [ ]:
# initialisations
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
sns.set(
    palette="Paired",
    #style='whitegrid',
    color_codes=True,
    rc={"figure.figsize": (12,8)}
)

# fairness metrics
from aif360.datasets import BinaryLabelDataset
from aif360.metrics import BinaryLabelDatasetMetric, ClassificationMetric

# model building
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix, accuracy_score, roc_auc_score, precision_score, recall_score, f1_score
from sklearn.metrics.pairwise import euclidean_distances
from sklearn.neighbors import NearestNeighbors
from sklearn.calibration import calibration_curve

import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Dropout, BatchNormalization, Activation, Dropout
from tensorflow.keras.callbacks import EarlyStopping, ReduceLROnPlateau
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.regularizers import L1L2
from tensorflow.keras.metrics import AUC, Precision, Recall, F1Score
from tensorflow.keras.losses import binary_crossentropy

# save results
import pickle

# suppress warnings
import warnings
warnings.filterwarnings("ignore")
import logging
logging.getLogger('matplotlib.font_manager').disabled = True

### Import the dataset

In [ ]:
from ucimlrepo import fetch_ucirepo

# fetch dataset
adult = fetch_ucirepo(id=2)

# data (as pandas dataframes)
X = adult.data.features
y = adult.data.targets

In [ ]:
# check to see if the data has imported correctly
X.head(4)

In [ ]:
# sort the target variables to be correct
y['income'] = y['income'].replace({'<=50K.': '<=50K', '>50K.': '>50K'})

### Preprocessing data

In [ ]:
# feature selection
X = X[['age', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country']]
X.dtypes

In [ ]:
X['workclass'] = X['workclass'].astype('category').cat.codes
X['education'] = X['education'].astype('category').cat.codes
X['marital-status'] = X['marital-status'].astype('category').cat.codes
X['occupation'] = X['occupation'].astype('category').cat.codes
X['relationship'] = X['relationship'].astype('category').cat.codes
X['race'] = X['race'].astype('category').cat.codes
X['sex'] = X['sex'].astype('category').cat.codes
X['native-country'] = X['native-country'].astype('category').cat.codes
y['income'] = y['income'].astype('category').cat.codes

In [ ]:
X = X.astype('float32')
y = y.astype('float32')

In [ ]:
y = y['income']

## FairVIC
Here you will find the code behind our method, FairVIC, please read through the comments for explanations 😀

### FairVIC class
Here is all the code, in one class, for training a custom neural network on our new loss function FairVIC.

In [ ]:
import numpy as np
import tensorflow as tf
from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, BatchNormalization, Activation, Dropout
from tensorflow.keras.regularizers import L2
from tensorflow.keras.models import Model
from tensorflow.keras.losses import binary_crossentropy
import matplotlib.pyplot as plt

from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

class FairModel():
    def __init__(self):
        self.model = self.create_model()
        self.bottleneck_extractor = tf.keras.Model(
            inputs=self.model.input,
            outputs=self.model.get_layer("bottleneck").output
        )
        self.protected_attribute = None
        self.privileged_groups = None
        self.unprivileged_groups = None
        self.favorable_label = None
        self.unfavorable_label = None

    def configure_fairness_evaluation(self, protected_attribute,
                                      privileged_groups, unprivileged_groups,
                                      favorable_label, unfavorable_label):
        self.protected_attribute = protected_attribute
        self.privileged_groups = privileged_groups
        self.unprivileged_groups = unprivileged_groups
        self.favorable_label = favorable_label
        self.unfavorable_label = unfavorable_label

    def create_model(self):
        """
        Creates a Functional model with a 2D 'bottleneck' layer.
        """
        input_layer = tf.keras.Input(shape=(X_train.shape[1],))

        # Encoder
        x = Dense(128, kernel_regularizer=L1L2(l1=0.0001, l2=0.0001), kernel_initializer='he_normal')(input_layer)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(0.25)(x)

        x = Dense(64, kernel_regularizer=L1L2(l1=0.0001, l2=0.0001), kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(0.25)(x)

        x = Dense(32, kernel_regularizer=L1L2(l1=0.0001, l2=0.0001), kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(0.25)(x)

        bottleneck = Dense(2, name="bottleneck")(x)

        # Decoder (mirrors the encoder)
        x = Dense(32, kernel_regularizer=L1L2(l1=0.0001, l2=0.0001), kernel_initializer='he_normal')(bottleneck)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(0.25)(x)

        x = Dense(64, kernel_regularizer=L1L2(l1=0.0001, l2=0.0001), kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)
        x = Dropout(0.25)(x)

        x = Dense(128, kernel_regularizer=L1L2(l1=0.0001, l2=0.0001), kernel_initializer='he_normal')(x)
        x = BatchNormalization()(x)
        x = Activation('relu')(x)

        output_layer = Dense(1, activation='sigmoid')(x)  # Final output

        # Create the model
        model = tf.keras.Model(inputs=input_layer, outputs=output_layer)
        return model

    def fairvic_wrapper(self,
                        inputs,
                        protected_column_idx,
                        protected_attribute,
                        lambda_binary,
                        lambda_cov,
                        lambda_var,
                        lambda_inv,
                        training=True,
                        eps=1e-8):
        """
        This wrapper returns a combined loss:
          - Standard binary crossentropy
          - Invariance loss (flip protected attribute)
          - Covariance loss
          - Representation variance penalty (replaces old group-based var)
        """

        def fairvic_loss(y_true, y_pred):
            # ====== 1) Binary Classification Loss ======
            y_pred_squeezed = tf.squeeze(y_pred, axis=1)
            binary_loss = binary_crossentropy(y_true, y_pred_squeezed)

            # ====== 2) Variance Loss ======
            variance_loss = tf.constant(0.0, dtype=tf.float32)
            if lambda_var > 0.0:
                embedding = self.bottleneck_extractor(inputs)
                embedding_std = tf.math.reduce_std(embedding, axis=0)
                gamma = 1.0
                variance_loss = tf.reduce_mean(tf.nn.relu(gamma - embedding_std))

            # ====== 3) Invariance Loss ======
            flipped_inputs = tf.identity(inputs)
            flipped_protected = 1 - tf.cast(inputs[:, protected_column_idx], tf.float32)
            flipped_inputs = tf.concat(
                [flipped_inputs[:, :protected_column_idx],
                 tf.expand_dims(flipped_protected, axis=1),
                 flipped_inputs[:, protected_column_idx + 1:]],
                axis=1
            )
            y_flip_pred = self.model(flipped_inputs, training=training)
            invariance_loss = tf.reduce_mean(tf.square(y_pred_squeezed - tf.squeeze(y_flip_pred)))

            # ====== 4) Covariance Loss ======
            y_pred_reshaped = tf.reshape(y_pred, (-1, 1))
            protected_reshaped = tf.reshape(protected_attribute, (-1, 1))
            cov_matrix = tf.matmul(
                tf.transpose(y_pred_reshaped - tf.reduce_mean(y_pred_reshaped, axis=0)),
                protected_reshaped
            )
            covariance_loss = tf.sqrt(tf.reduce_sum(tf.square(cov_matrix))) / \
                              tf.cast(tf.shape(y_pred_reshaped)[0], tf.float32)

            # ====== 5) Combine all into total loss ======
            total_loss = (lambda_binary * binary_loss
                          + lambda_inv * invariance_loss
                          + lambda_cov * covariance_loss
                          + lambda_var * variance_loss)
            return total_loss

        return fairvic_loss

    @tf.function
    def train_step(self, inputs, labels, protected, optimizer,
                   lambda_binary, lambda_cov, lambda_var, lambda_inv):
        """
        Single training step with gradient update.
        """
        with tf.GradientTape() as tape:
            predictions = self.model(inputs, training=True)
            loss = self.fairvic_wrapper(
                inputs, self.protected_column_idx, protected,
                lambda_binary, lambda_cov, lambda_var, lambda_inv,
                training=True)(labels, predictions)

        gradients = tape.gradient(loss, self.model.trainable_variables)
        optimizer.apply_gradients(zip(gradients, self.model.trainable_variables))
        return loss

    @tf.function
    def val_step(self, inputs, labels, protected,
                 lambda_binary, lambda_cov, lambda_var, lambda_inv):
        """
        Validation step.
        """
        predictions = self.model(inputs, training=False)
        loss = self.fairvic_wrapper(
            inputs, self.protected_column_idx, protected,
            lambda_binary, lambda_cov, lambda_var, lambda_inv,
            training=False
        )(labels, predictions)
        return loss

    def train(self, epochs, batch_size,
              X_train, y_train, X_train_protected,
              X_val,   y_val,   X_val_protected,
              lambda_binary, lambda_cov, lambda_var, lambda_inv,
              protected_column_idx):
        """
        Main training loop.
        """

        self.protected_column_idx = protected_column_idx

        train_dataset = tf.data.Dataset.from_tensor_slices(
            (X_train, y_train, X_train_protected)
        ).batch(batch_size)
        val_dataset = tf.data.Dataset.from_tensor_slices(
            (X_val, y_val, X_val_protected)
        ).batch(batch_size)

        optimizer = tf.keras.optimizers.Adam(learning_rate=0.05)
        epoch_train_losses = []
        epoch_val_losses = []

        for epoch in range(epochs):
            print(f"Epoch {epoch+1}/{epochs}")
            train_losses_batch = []
            for batch, (inputs_batch, labels_batch, protected_batch) in enumerate(train_dataset):
                loss_val = self.train_step(inputs_batch, labels_batch, protected_batch,
                                           optimizer,
                                           lambda_binary, lambda_cov, lambda_var, lambda_inv)
                train_losses_batch.append(loss_val.numpy())
            train_loss = np.mean(train_losses_batch)

            val_losses_batch = []
            for batch, (inputs_batch, labels_batch, protected_batch) in enumerate(val_dataset):
                loss_val = self.val_step(inputs_batch, labels_batch, protected_batch,
                                         lambda_binary, lambda_cov, lambda_var, lambda_inv)
                val_losses_batch.append(loss_val.numpy())
            val_loss = np.mean(val_losses_batch)

            print(f"Epoch {epoch+1}, Loss: {train_loss:.4f}, Val Loss: {val_loss:.4f}, Lr: {optimizer.learning_rate.numpy():.5f}")
            epoch_train_losses.append(train_loss)
            epoch_val_losses.append(val_loss)

        self.plot_losses(epoch_train_losses, epoch_val_losses)

    def plot_losses(self, train_losses, val_losses):
        """
        Plot the training and validation losses to monitor overfitting.
        """
        plt.title('Learning Curves')
        plt.xlabel('Epoch')
        plt.ylabel('Loss')
        plt.plot(train_losses, label='train')
        plt.plot(val_losses, label='val')
        plt.legend()
        plt.show()

    def evaluate(self, X_test, y_test):
        """
        Evaluate the trained model using standard classification metrics.
        """
        predictions = self.model.predict(X_test)
        predictions_binary = (predictions > 0.5).astype(int)

        acc = accuracy_score(y_test, predictions_binary)
        auc = roc_auc_score(y_test, predictions)
        precision = precision_score(y_test, predictions_binary)
        recall = recall_score(y_test, predictions_binary)
        f1 = f1_score(y_test, predictions_binary)

        return {
            'acc': acc,
            'auc': auc,
            'precision': precision,
            'recall': recall,
            'f1': f1,
            'y_test': y_test,
            'y_pred': predictions_binary
        }

### Using FairVIC
Please follow the walkthrough below on how to use FairVIC

Firstly, we want to split out data in training, validation, and test datasets

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)
X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2)

In [ ]:
# creating copies for counterfactual evaluation later
X_copy = X.copy()
X_train_copy = X_train.copy()
X_test_copy = X_test.copy()
X_val_copy = X_val.copy()

Next, we want to extract the pritected characteristic column from the dataset. In this psuedo-example, we extract only the 'sex' column

In [ ]:
protected_attribute_column_name = 'sex'
protected_column_idx = X.columns.get_loc(protected_attribute_column_name)
X_train_protected = X.loc[X_train.index, protected_attribute_column_name]
X_test_protected = X.loc[X_test.index, protected_attribute_column_name]
X_val_protected = X.loc[X_val.index, protected_attribute_column_name]

We then can do some standard scaling of the data which is typical for most neural net training

In [ ]:
scaler = MinMaxScaler()
X_train = scaler.fit_transform(X_train)
X_test = scaler.transform(X_test)
X_val = scaler.transform(X_val)

Here we can define the weights for each term in the loss function. This can be done directly, as in the comments, or this can be done by changing a single accuracy loss value, which will then give the remaining terms equal weights.

In [ ]:
#lambda_binary = 0.1
#lambda_var = 0.1
#lambda_inv = 0.1
#lambda_cov = 0.7

acc_weight = 0.1
lambda_binary = acc_weight
lambda_cov = lambda_var = lambda_inv = (1-acc_weight)/3

Next, we need to define the model. Here we can just call a new instance of the class and it will instantiate a new neural network that has been created using the 'create_model()' function

In [ ]:
fair_model = FairModel()

Now we have everything we need to train the model. In the class you can change the weights of the loss function multiplers by tweaking the variables below. 'lambda_binary' in this case is simply our accuracy loss that is used for this example, this can be replaced by any other accuracy loss function. We will use 200 training epochs and a batch size of 256 right now.


In [ ]:
fair_model.train(200, 256,
                 X_train, y_train, X_train_protected,
                 X_val, y_val, X_val_protected,
                 lambda_binary=lambda_binary,
                 lambda_cov=lambda_cov,
                 lambda_var=lambda_var,
                 lambda_inv=lambda_inv,
                 protected_column_idx=protected_column_idx)

### Evaluating FairVIC

To evaluate the model's performance in standard accuracy metrics, we have provided a function below to do so. This can obviously be changed to allow for any custom metrics needed.

In [ ]:
eval_metrics = fair_model.evaluate(X_test, y_test)

To evaluate the model's performance in group fairness metrics, we use this little code snippet below. All that is needed is to specifiy the 'protected attribute' as well as the 'priviledged' and 'unprivileged' groups.

In [ ]:
X_test = pd.DataFrame(data=X_test, columns=['age', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'])
predictions = eval_metrics['y_pred']

test_df = pd.concat([X_test.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)
test_df.columns = list(X_test.columns) + ['label']

test_df = test_df.astype(int)

test_bld = BinaryLabelDataset(df=test_df,
                              label_names=['label'],
                              protected_attribute_names=['sex'],
                              favorable_label=1,
                              unfavorable_label=0)

predictions_bld = test_bld.copy()
predictions_bld.labels = predictions.reshape(-1, 1)

classification_metric = ClassificationMetric(test_bld,
                                              predictions_bld,
                                              unprivileged_groups=[{'sex': 0}],
                                              privileged_groups=[{'sex': 1}])

fairness_metrics = {
    'disparate_impact': classification_metric.disparate_impact(),
    'equalized_odds_difference': classification_metric.equalized_odds_difference(),
    'average_abs_odds_difference': classification_metric.average_abs_odds_difference(),
    'statistical_parity_difference': classification_metric.mean_difference(),
    'error_rate_ratio': classification_metric.error_rate_ratio(),
    'theil_index': classification_metric.theil_index(),
    'consistency': classification_metric.consistency(),
    'ppv_p': classification_metric.positive_predictive_value(privileged=True),
    'ppv_u': classification_metric.positive_predictive_value(privileged=False),
    'npv_p': classification_metric.negative_predictive_value(privileged=True),
    'npv_u': classification_metric.negative_predictive_value(privileged=False),
    'fdr_p': classification_metric.false_discovery_rate(privileged=True),
    'fdr_u': classification_metric.false_discovery_rate(privileged=False),
    'for_p': classification_metric.false_omission_rate(privileged=True),
    'for_u': classification_metric.false_omission_rate(privileged=False),
    'tpr_p': classification_metric.true_positive_rate(privileged=True),
    'tpr_u': classification_metric.true_positive_rate(privileged=False),
    'fpr_p': classification_metric.false_positive_rate(privileged=True),
    'fpr_u': classification_metric.false_positive_rate(privileged=False)
    }

Finally, to see our results for both accuracy and fairness, we print every metric here!

In [ ]:
print(f"Results")
print("============================================================================================")
print(f"Accuracy: {eval_metrics['acc']}")
print(f"AUC: {eval_metrics['auc']}")
print(f"Precision: {eval_metrics['precision']}")
print(f"Recall: {eval_metrics['recall']}")
print(f"F1 Score: {eval_metrics['f1']}")
print("---------------------------------------------")
print("Fairness Metrics:")
print(f"Disparate Impact: {fairness_metrics['disparate_impact']}")
print(f"Equalized Odds Difference: {fairness_metrics['equalized_odds_difference']}")
print(f"Average Absolute Odds Difference: {fairness_metrics['average_abs_odds_difference']}")
print(f"Statistical Parity Difference: {fairness_metrics['statistical_parity_difference']}")
print(f"Error Rate Ratio: {fairness_metrics['error_rate_ratio']}")
print(f"Theil Index: {fairness_metrics['theil_index']}")
print(f"Consistency: {fairness_metrics['consistency']}")
print("---------------------------------------------")
print("For the Privileged Group:")
print(f"Positive Predictive Value: {fairness_metrics['ppv_p']}")
print(f"Negative Predictive Value: {fairness_metrics['npv_p']}")
print(f"False Discovery Rate: {fairness_metrics['fdr_p']}")
print(f"False Omission Rate: {fairness_metrics['for_p']}")
print(f"True Positive Rate: {fairness_metrics['tpr_p']}")
print(f"False Positive Rate: {fairness_metrics['fpr_p']}")
print("---------------------------------------------")
print("For the Unprivileged Group:")
print(f"Positive Predictive Value: {fairness_metrics['ppv_u']}")
print(f"Negative Predictive Value: {fairness_metrics['npv_u']}")
print(f"False Discovery Rate: {fairness_metrics['fdr_u']}")
print(f"False Omission Rate: {fairness_metrics['for_u']}")
print(f"True Positive Rate: {fairness_metrics['tpr_u']}")
print(f"False Positive Rate: {fairness_metrics['fpr_u']}")
print("============================================================================================")

#### Counterfactual Fairness Evaulation
To assess for individual fairness further, we train a counterfactual model where the protected attribute column has its values switched.

In [ ]:
# function to create a counterfactual dataset by flipping the 'sex' attribute
def flip_attribute(data):
    data = pd.DataFrame(data=data, columns=['age', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'])
    data['sex'] = 1 - data['sex']
    return data

# create counterfactual versions for training, testing, and validation sets
X_cf = flip_attribute(X_copy)
X_train_cf = flip_attribute(X_train_copy)
X_test_cf = flip_attribute(X_test_copy)
X_val_cf = flip_attribute(X_val_copy)

protected_attribute_column_name = 'sex'
X_train_protected_cf = X_cf.loc[X_train_cf.index, protected_attribute_column_name]
X_test_protected_cf = X_cf.loc[X_test_cf.index, protected_attribute_column_name]
X_val_protected_cf = X_cf.loc[X_val_cf.index, protected_attribute_column_name]

scaler = MinMaxScaler()
X_train_cf = scaler.fit_transform(X_train)
X_test_cf = scaler.transform(X_test)
X_val_cf = scaler.transform(X_val)

In [ ]:
# initialize the counterfactual model to compare against our fairmodel
counterfactual_model = FairModel()

# train fairvic on the counterfactual data
counterfactual_model.train(200, 256,
                           X_train, y_train, X_train_protected,
                           X_val, y_val, X_val_protected,
                           lambda_binary=lambda_binary,
                           lambda_cov=lambda_cov,
                           lambda_var=lambda_var,
                           lambda_inv=lambda_inv,
                           protected_column_idx=protected_column_idx)

In [ ]:
# evaulate the counterfactual model
eval_metrics_cf = counterfactual_model.evaluate(X_test_cf, y_test)

In [ ]:
X_test_cf = pd.DataFrame(data=X_test_cf, columns=['age', 'workclass', 'education', 'marital-status', 'occupation', 'relationship', 'race', 'sex', 'capital-gain', 'capital-loss', 'hours-per-week', 'native-country'])
predictions_cf = eval_metrics['y_pred']

test_df_cf = pd.concat([X_test_cf.reset_index(drop=True), y_test.reset_index(drop=True)], axis=1)
test_df_cf.columns = list(X_test_cf.columns) + ['label']

test_df_cf = test_df_cf.astype(int)

test_bld_cf = BinaryLabelDataset(df=test_df_cf,
                              label_names=['label'],
                              protected_attribute_names=['sex'],
                              favorable_label=1,
                              unfavorable_label=0)

predictions_bld_cf = test_bld_cf.copy()
predictions_bld_cf.labels = predictions_cf.reshape(-1, 1)

classification_metric = ClassificationMetric(test_bld_cf,
                                              predictions_bld_cf,
                                              unprivileged_groups=[{'sex': 0}],
                                              privileged_groups=[{'sex': 1}])

fairness_metrics_cf = {
        'equalized_odds_difference': classification_metric.equalized_odds_difference(),
        'average_abs_odds_difference': classification_metric.average_abs_odds_difference(),
        'disparate_impact': classification_metric.disparate_impact(),
        'demographic_parity_difference': classification_metric.mean_difference(),
        'ppv_p': classification_metric.positive_predictive_value(privileged=True),
        'npv_p': classification_metric.negative_predictive_value(privileged=True),
        'fdr_p': classification_metric.false_discovery_rate(privileged=True),
        'for_p': classification_metric.false_omission_rate(privileged=True),
        'ppv_u': classification_metric.positive_predictive_value(privileged=False),
        'npv_u': classification_metric.negative_predictive_value(privileged=False),
        'fdr_u': classification_metric.false_discovery_rate(privileged=False),
        'for_u': classification_metric.false_omission_rate(privileged=False),
    }

In [ ]:
print(f"Results for counterfactual model")
print("============================================================================================")
print(f"Accuracy: {eval_metrics['acc']}")
print(f"AUC: {eval_metrics['auc']}")
print(f"Precision: {eval_metrics['precision']}")
print(f"Recall: {eval_metrics['recall']}")
print(f"F1 Score: {eval_metrics['f1']}")
print("---------------------------------------------")
print("Fairness Metrics:")
print(f"Disparate Impact: {fairness_metrics['disparate_impact']}")
print(f"Equalized Odds Difference: {fairness_metrics['equalized_odds_difference']}")
print(f"Average Absolute Odds Difference: {fairness_metrics['average_abs_odds_difference']}")
print(f"Statistical Parity Difference: {fairness_metrics['statistical_parity_difference']}")
print(f"Error Rate Ratio: {fairness_metrics['error_rate_ratio']}")
print(f"Theil Index: {fairness_metrics['theil_index']}")
print(f"Consistency: {fairness_metrics['consistency']}")
print("---------------------------------------------")
print("For the Privileged Group:")
print(f"Positive Predictive Value: {fairness_metrics['ppv_p']}")
print(f"Negative Predictive Value: {fairness_metrics['npv_p']}")
print(f"False Discovery Rate: {fairness_metrics['fdr_p']}")
print(f"False Omission Rate: {fairness_metrics['for_p']}")
print(f"True Positive Rate: {fairness_metrics['tpr_p']}")
print(f"False Positive Rate: {fairness_metrics['fpr_p']}")
print("---------------------------------------------")
print("For the Unprivileged Group:")
print(f"Positive Predictive Value: {fairness_metrics['ppv_u']}")
print(f"Negative Predictive Value: {fairness_metrics['npv_u']}")
print(f"False Discovery Rate: {fairness_metrics['fdr_u']}")
print(f"False Omission Rate: {fairness_metrics['for_u']}")
print(f"True Positive Rate: {fairness_metrics['tpr_u']}")
print(f"False Positive Rate: {fairness_metrics['fpr_u']}")
print("============================================================================================")

That is how you can incorporate FairVIC into a neural network's training. It is elegant yet efficient. We encourage you to mess around with the weights! 😀